In [30]:
import pymongo
from elasticsearch import Elasticsearch

In [31]:
db = pymongo.MongoClient('10.18.6.46',27001)

In [32]:
es = Elasticsearch(['10.18.6.102:9200'])

In [11]:
doc = db['db_parker']['jsl']

In [12]:
item = doc.find_one()

In [13]:
item

{'_id': ObjectId('5c249f29c4c05d4ba4bfa49d'),
 'creator': 'greatbear',
 'title': '各位研究技术指标有什么心得体会？',
 'content': '各位研究技术指标多久了？研究这东西，能帮炒股赚钱么：）',
 'resp_no': 11,
 'createTime': '2018-12-14 08:38',
 'crawlTime': '2019-03-16 09:54:43',
 'url': 'https://www.jisilu.cn/question/297952',
 'resp': [{'seeker24680_0': ['0', '可以说很多不明觉厉的词，适合用于忽悠别人，具体效果同风水先生。']},
  {'joyfulli_1': ['0',
    '首先要知道技术指标是怎么来的\n对过去的数据进行统计分析，然后进行预测\n说白了就是统计学\n所以，学过统计学就知道技术分析怎么用了']},
  {'老高0813_2': ['0', '基本没卵用']},
  {'z383788052_3': ['0', '从来只看指标，只看k']},
  {'花园小琴_4': ['0', '看图形，一般只看低买高卖，但事后后悔，赚少了']},
  {'风险搬运工_5': ['0', '趋势（多和空）和震荡结合得做（同时做），不做单边、所谓轮动策略。\n倾向于胜率低，赔率高的策略。']},
  {'smag_6': ['0', '基本不看指标，静下心感受波动，在波动中下注。。。']},
  {'海浪头头_7': ['0', '同意美棠子的看法']},
  {'jsl0900_8': ['0', '从技术指标上看，我可以预测所有交易标的的后续走势，至于准确不准确，那不是我关心的问题']},
  {'美棠子_9': ['4', '只是个辅助工具，基本面是核心，切不可颠倒主次。']},
  {'蔓越橘_10': ['0', '用于T一下可以，其他就算了吧。']}],
 'last_resp_date': '2018-12-14 10:14'}

In [14]:
keys= item.keys()

In [15]:
keys

dict_keys(['_id', 'creator', 'title', 'content', 'resp_no', 'createTime', 'crawlTime', 'url', 'resp', 'last_resp_date'])

In [34]:
save_db = db['db_parker']['jsl_note']

In [24]:
import datetime

In [37]:
url_set = set()
for item in doc.find():
    creator=item.get('creator')
    title=item.get('title')
    content=item.get('content')
    resp_no=item.get('resp_no')
    createTime=item.get('createTime')
    
    url=item.get('url')
    if url in url_set:
        continue
    else:
        url_set.add(url)
    
    if createTime is None:
        save_db.insert_one({'url':url})
        continue
        
    createTime=datetime.datetime.strptime(createTime,'%Y-%m-%d %H:%M').strftime('%Y-%m-%dT%H:%M:%S')

    crawlTime=item.get('crawlTime')
    crawlTime=datetime.datetime.strptime(crawlTime,'%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')

    
    resp_list=item.get('resp')
    last_resp_date=item.get('last_resp_date')
    last_resp_date=datetime.datetime.strptime(last_resp_date,'%Y-%m-%d %H:%M').strftime('%Y-%m-%dT%H:%M:%S')

    _resp_list=[]
    for sub_resp in resp_list:
        resp_author = list(sub_resp.keys())[0]
        _resp_author=''.join(resp_author.split('_')[:-1])
        agree=sub_resp[resp_author][0]
        resp_content=sub_resp[resp_author][1]
        d={}
        d['resp_agree']=int(agree)
        d['resp_author']=_resp_author
        d['resp_content']=resp_content

        _resp_list.append(d)
    # last_resp_date=item.get('last_resp_date')

    body = {
    'creator':creator,
    'title':title,
    'content':content,
    'resp_no':int(resp_no),
    'createTime':createTime,
    'crawlTime':crawlTime,
    'url':url,
    'resp':_resp_list,
    'last_resp_date':last_resp_date
    }

    es.index(index='jsl',doc_type='doc',body=body)

AutoReconnect: 10.18.6.46:27001: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

In [23]:
crawlTime

'2019-03-16 09:54:43'

In [19]:
_resp_list

[{'resp_agree': 0,
  'resp_author': 'seeker24680',
  'resp_content': '可以说很多不明觉厉的词，适合用于忽悠别人，具体效果同风水先生。'},
 {'resp_agree': 0,
  'resp_author': 'joyfulli',
  'resp_content': '首先要知道技术指标是怎么来的\n对过去的数据进行统计分析，然后进行预测\n说白了就是统计学\n所以，学过统计学就知道技术分析怎么用了'},
 {'resp_agree': 0, 'resp_author': '老高0813', 'resp_content': '基本没卵用'},
 {'resp_agree': 0, 'resp_author': 'z383788052', 'resp_content': '从来只看指标，只看k'},
 {'resp_agree': 0,
  'resp_author': '花园小琴',
  'resp_content': '看图形，一般只看低买高卖，但事后后悔，赚少了'},
 {'resp_agree': 0,
  'resp_author': '风险搬运工',
  'resp_content': '趋势（多和空）和震荡结合得做（同时做），不做单边、所谓轮动策略。\n倾向于胜率低，赔率高的策略。'},
 {'resp_agree': 0,
  'resp_author': 'smag',
  'resp_content': '基本不看指标，静下心感受波动，在波动中下注。。。'},
 {'resp_agree': 0, 'resp_author': '海浪头头', 'resp_content': '同意美棠子的看法'},
 {'resp_agree': 0,
  'resp_author': 'jsl0900',
  'resp_content': '从技术指标上看，我可以预测所有交易标的的后续走势，至于准确不准确，那不是我关心的问题'},
 {'resp_agree': 4,
  'resp_author': '美棠子',
  'resp_content': '只是个辅助工具，基本面是核心，切不可颠倒主次。'},
 {'resp_agree': 0, 'resp_author': '蔓越橘', 

In [ ]:
for item in doc.find_one():


In [ ]:
db